# Text Summarization Overview

For modelling, I perform both extractive and abstractive summarization. For extractive summarization, I use the BERT transformer model and customize it to use the pre-trained weights of the **`sciBERT`** model which specializes in scientific texts, which fit our purpose. For every text, I determine the optimal number of sentences for the extracted summary.

For abstractive summarization, I first concatenate the abstract, extractive summary, and conclusion together since much of the important information can be found in them. Then, I use the **`facebook-BART-large-cnn`** transformer model to perform the abstraction.

Before moving on to our summarization, I'll first explore some important data characteristics like the average word length and token length per text.

See this reference [article](https://towardsdatascience.com/beginners-guide-for-data-cleaning-and-feature-extraction-in-nlp-756f311d8083) for a basic introduction to NLP feature extraction.

In order to perform the feature extraction, I created a `class TextSummarizer` which serves both as the feature extractor and the text summarizer. The class has the following functions and their corresponding capabilities: 
* `avg_word`: Function for getting the average length of a word in a text
* `count_punctuation`: Function for getting the number of punctuations in a text
* `get_optimal_number_sentences`: Function for getting the optimal number of sentences for extractive summarization
* `extract_text_features`: Function which returns the following features number of stopwords, punctuations, numerical characters, words, average word length and stopwords to word ratio. 
* `extractive_summarizer`: Function for performing extractive text summarization
* `join_extracted_summary`: Function for concatenating the abstract, extractive text summary, and conclusion
* `abstractive_summarizer`: Function for performing abstractive summarization with the text. 

Using this class, I perform all the necessary explorations, and then directly proceed to performing extractive text summarization and abstractive text summarization with BERT and BART respectively. 

For extractive text summarization, I used the following references:
* [bert-extractive-summarizer](https://github.com/dmmiller612/bert-extractive-summarizer)
* [Handling coreference resolution with Python](https://kaveeshabaddage.medium.com/how-to-resolve-coreference-resolution-using-python-97fcd6b2cedb) 
* [sciBERT](https://github.com/allenai/scibert)

*Note: Due to the lack of computational resources, coreference handling could not be applied as the memory provided by Google Collab is not enough and leads to a session crash.*

For abstractive text summarization, I used the following reference:
* [facebook-bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)

# Building the TextSummarizer class

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install tensorflow-gpu # For CPMTokenizer
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 35.0 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 6.6 MB 60.0 MB/s 
     |████████████████████████████████| 101 kB 14.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 1.6 MB 56.4 MB/s 
     |████████████████████████████████| 438 kB 73.2 MB/s 
     |████████████████████████████████| 5.8 MB 44.8 MB/s 
  Attempting uninstall: tensorf

In [2]:
import pandas as pd
import numpy as np

# Data preprocessing
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Text Summarization
from transformers import *
from summarizer import Summarizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
class TextSummarizer:
  def __init__(self, data):
    self.data = data

  # Helper functions

  # Get average word length in a document
  def avg_word(self, data):
    words = data.split()
    length = (sum(len(word) for word in words)/(len(words)+0.000001))

    return length
  
  # Get number of punctuations in a document
  def count_punctuation(self, data):
    punctuation_count = sum([1 for char in data if char in string.punctuation])

    return punctuation_count
  
  # Get optimal number of sentences for extractive summarization
  def get_optimal_number_sentences(self, data, model):

    optimal_num_sentences = model.calculate_optimal_k(data, k_max=10)

    return optimal_num_sentences
  
  # Extract numerical text features
  def extract_text_features(self, text_column):
    
    """
    Extracts text features such as number of stopwords, punctuations,
    numerical characters, average word length, average document length
    :param text_column: dataframe column to perform feature extraction on
    :return: dataframe with new feature columns
    """
    
    # Get number of stop words
    stop_words = stopwords.words('english')
    self.data["num_stopwords"] = self.data[text_column].apply(lambda x: 
    len([x for x in x.split() if x in stop_words]))

    # Get number of punctuations
    self.data["num_punctuations"] = self.data[text_column].apply(lambda x: 
    self.count_punctuation(x))

    # Get number of numerical characters
    self.data["num_numerics"] = self.data[text_column].apply(lambda x:
    len([x for x in x.split() if x.isdigit()]))

    # Get number of words in the document
    self.data["num_words"] = self.data[text_column].apply(lambda x: 
    len(str(x).split(" ")))

    # Get average word length in document

    self.data["avg_word_length"] = self.data[text_column].apply(lambda x: 
    round(self.avg_word(x),1))

    # Get the stopwords to word ratio
    self.data["stopwords_to_words_ratio"] = round(self.data["num_stopwords"] / self.data["num_words"], 3)

    return self.data
  
  def extractive_summarizer(self, model, text_column):
    
    """
    Performs extractive text summarization with BERT and allows for different 
    pretrained model loading and configurations.
    :param model: initialized pretrained model
    :param text_column: dataframe column to perform text_summarization on
    :return: dataframe with summarized text columns
    """

    self.data["extractive_summarized_text"] = self.data[text_column].apply(lambda x:
    "".join(model(x, num_sentences=self.get_optimal_number_sentences(x, model))))

    return self.data   


  def join_extracted_summary(self, abstract, extracted_summary, conclusion):

    """
    Concatenates the abstract, extractive_summarized_text, and conclusion columns
    into one column for abstractive summarization
    :param abstract: abstract column
    :param extracted_summary: extractive_summarized_text column
    :param conclusion: conclusion column
    :return: dataframe with concatenated abstract, extracted summary and conclusion 
    columns
    """

    self.data["combined_text"] = self.data[[abstract, extracted_summary, conclusion]].agg(
        " ".join, axis=1
    )

    return self.data

  def abstractive_summarizer(self, model, text_column, max_length=750, min_length=250):
    
    """
    Performs abstract text summarization with BART using the extracted summary combined
    with the abstract and conclusion of the text.
    :param model: pipeline of the abstractive summarizer model
    :param text_column: dataframe column to perform text_summarization on
    :return: dataframe with summarized text columns
    """

    summaries_list = []
    for i in range(len(self.data[text_column])):
      text = self.data[text_column][i]
      try:
        summary = model(text, max_length = max_length, 
        min_length = min_length, do_sample=False)[-1]["summary_text"]
      except:
        # Decrease the length of the token to 1024 if it exceeds
        text = text[:1024]
        summary = model(text, max_length = max_length, 
        min_length = min_length, do_sample=False)[-1]["summary_text"]
      
      summaries_list.append(summary)
    
    self.data["abstractive_summaries"] = summaries_list
      
    return self.data

# Performing Text Summarization

## Extractive Summarization

In [5]:
text_df = pd.read_csv("top5_cleaned.csv")
text_df.drop("Unnamed: 0", axis=1, inplace=True)
text_class = TextSummarizer(text_df)

# Extract all features from the combined abstract, body, and conclusion text
text_class.extract_text_features("full_text")

# Use scibert to perform extractive summarization
pretrained_model = 'allenai/scibert_scivocab_uncased'

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained(pretrained_model)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
custom_model = AutoModel.from_pretrained(pretrained_model, config=custom_config)

# Create pretrained-model object
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

# Extractive summarization
extractive_summarized_text = text_class.extractive_summarizer(model, "full_text")

# Optional: Save dataframe containing extractive summaries
extractive_summarized_text.to_csv("extractive_summarized_dataframe_final.csv")

# Check extractive summaries
extractive_summaries = extractive_summarized_text["extractive_summarized_text"]
print(extractive_summaries)

https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpa002tp2s


storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
creating metadata file for /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
creating metadata file for /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingf

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
creating metadata file for /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
loading weights file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/de14937a851e8180a2bc5660c0041d385f8a0c62b1b2ccafa46df31043a2390c.74830bb01a0ffcdeaed8be9916312726d0c4cd364ac6fc15b375f789eaff4cbb
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictio

0    Word lattice decoding has proven useful in spo...
1    We formulate the problem of nonprojective depe...
2    We propose a cascaded linear model for joint C...
3    In this paper, we propose a novel string-todep...
4    We present a novel transition system for depen...
Name: extractive_summarized_text, dtype: object


# Abstractive Summarization

Now, we perform abstractive summarization on the extractively summarized text

In [6]:
text_class = TextSummarizer(extractive_summarized_text)

# Concatenate the extractive summary with the abstract and conclusion
text_class.join_extracted_summary("abstract", "extractive_summarized_text", "conclusion")

# Instantiate abstractive summarizer model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
abstractive_summarized_text = text_class.abstractive_summarizer(summarizer, "combined_text")

# Save to csv
abstractive_summarized_text.to_csv("abstractive_summarized_dataframe_final.csv")
abstractive_summaries = abstractive_summarized_text["abstractive_summaries"]

# Compare abstractive summaries and full text
print(abstractive_summarized_text[["full_text", "abstractive_summaries"]])

https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpyogyz7c2


storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
creating metadata file for /root/.cache/huggingface/transformers/199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
loading configuration file https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dro

storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300
creating metadata file for /root/.cache/huggingface/transformers/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300
loading weights file https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of th

storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/4d8eeedc3498bc73a4b72411ebb3219209b305663632d77a6f16e60790b18038.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/4d8eeedc3498bc73a4b72411ebb3219209b305663632d77a6f16e60790b18038.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/facebook/bart-large-cnn/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn00d23p0


storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/0ddddd3ca9e107b17a6901c92543692272af1c3238a8d7549fa937ba0057bbcf.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/0ddddd3ca9e107b17a6901c92543692272af1c3238a8d7549fa937ba0057bbcf.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/bart-large-cnn/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgn84xc1d


storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/55c96bd962ce1d360fde4947619318f1b4eb551430de678044699cbfeb99de6a.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/55c96bd962ce1d360fde4947619318f1b4eb551430de678044699cbfeb99de6a.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/facebook/bart-large-cnn/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/4d8eeedc3498bc73a4b72411ebb3219209b305663632d77a6f16e60790b18038.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/facebook/bart-large-cnn/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/0ddddd3ca9e107b17a6901c92543692272af1c3238a8d7549fa937ba0057bbcf.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading fi

                                           full_text  \
0  Word lattice decoding has proven useful in spo...   
1  We formulate the problem of nonprojective depe...   
2  We propose a cascaded linear model for joint C...   
3  In this paper, we propose a novel string-todep...   
4  We present a novel transition system for depen...   

                               abstractive_summaries  
0  We show that prior work in translating lattice...  
1  We formulate the problem of nonprojective depe...  
2  We propose a cascaded linear model for joint C...  
3  In this paper, we propose a novel string-todep...  
4  System constructs arcs only between adjacent w...  


Now we know that the pipeline works and we can use it on the entire dataset!